In [13]:
class PowerQuandle():
    
    # elements X is a list of something, rhd_el is function X x X -> X and pow_el is a function X x Z -> X
    def __init__(self, elements, rhd_el, pow_el, one_el):
        self.elements = elements
        self.rhd_el = rhd_el
        self.pow_el = pow_el
        self.one_el = one_el
        
        self.one_i0 = elements.index(one_el)
        self.one_i1 = self.one_i0 + 1
    
    def rhd_i0(self, i, j):
        x = self.elements[i]
        y = self.elements[j]
        z = self.rhd_el(x, y)
        k = self.elements.index(z)
        return k
    
    def pow_i0(self, i, n):
        x = self.elements[i]
        y = self.pow_el(x, n)
        k = self.elements.index(y)
        return k
    
    def rhd_i1(self, i, j):
        return self.rhd_i0(i - 1, j - 1) + 1
    
    def pow_i1(self, i, n):
        return self.pow_i0(i - 1, n) + 1
    
    def create_precomputed_iso_pq(self, assume_id_periodic = True, power_range = None):
        # 0-indexed, it's just a choice
        nelements = list(IntegerRange(len(self.elements)))
        
        nrhddata = [[self.rhd_i0(i, j) for j in nelements] for i in nelements]
        
        def create_pow_memo_func(i):
            if assume_id_periodic:
                
                pows = []
                
                # n is not an element, just a power.
                for n in nelements:
                    j = self.pow_i0(i, n)
                    if j == self.one_i0 and n > 0:
                        break
                    pows.append(j)
                
                p = len(pows)
                
                return lambda n: pows[n % p]
                
            else:
                raise NotYetImplementedError()
                #if power_range == None:
                #    raise ValueError("power_range may not be None when assume_id_periodic is False!")
                
                #pows = dict()
                #for n in power_range:
                #    pows.put(n, ) # ITS NOT PUT
                    
        
        npowdata = [create_pow_memo_func(i) for i in nelements]
        
        nrhd = lambda i, j: nrhddata[i][j]
        
        npow = lambda i, n: npowdata[i](n)
        
        return PowerQuandle(nelements, nrhd, npow, self.one_i0)
        
    

In [2]:
def get_gap_group(n, i):
    return PermutationGroup(gap_group = gap.Image(gap.IsomorphismPermGroup(gap.SmallGroup(n, i)))).as_finitely_presented_group()

In [3]:
def list_gap_groups(n):
    for i, H in enumerate(gap.AllSmallGroups(n)):
        print(i + 1, H.StructureDescription())

In [4]:
def Pq(G):
    return PowerQuandle(list(G), lambda x, y: x * y * x^-1, lambda x, n: x ^ n, G.one())

In [27]:
def PqGap(G):
    return PowerQuandle(list(G.AsList()), lambda x, y: x * y * x^-1, lambda x, n: x ^ n, G.One())

In [5]:
# Assumes powers are cyclic. Could we deduce this generally?
def Gr(pq, assume_cyclic = True, pow_ranges = None):
    N = len(pq.elements)
    
    Is = IntegerRange(1, N + 1)
    
    
    one_rel = [[pq.one_i1]]
    
    rhd_rels = []
    for i in Is:
        for j in Is:
            rhd_rels.append([i, j, -i, -pq.rhd_i1(i, j)])
    
    
    pow_rels = []
    if assume_cyclic:
        for i in Is:
            # n is not an element of the power quandle, it is just practical to use since the order of an element is at most the order of the group.
            for n in Is:
                j = pq.pow_i1(i, n)
                pow_rels.append(n * [i] + [-j])
                if j == pq.one_i1:
                    break
    else:
        if pow_ranges == None:
            raise ValueError("If not assume cyclic, then a power range must be specified")
        
        for i in Is:
            for n in pow_ranges:
                j = pq.pow_i1(i, n)
                pow_rels.append(n * [i] + [-j])
    
    rels = one_rel + rhd_rels + pow_rels
        
    FG = FreeGroup(N)
    FGR = FG / rels
    FGS = FGR.simplified()
    return FGS

In [6]:
#list_gap_groups(8)

1 C8
2 C4 x C2
3 D8
4 Q8
5 C2 x C2 x C2


In [7]:
#C8 = get_gap_group(8, 1); C8.structure_description()

'C8'

In [8]:
#G = Gr(Pq(C8))

In [9]:
#G.is_isomorphic(C8)

True

In [10]:
#G.structure_description()

'C8'

In [14]:
def test_memo_pq(pq):
    
    pq1 = pq.create_precomputed_iso_pq()
    
    N = len(pq.elements)
    
    assert len(pq1.elements) == N
    
    assert pq.one_i0 == pq1.one_el
    assert pq.one_i0 == pq1.one_i0
    
    for i in IntegerRange(N):
        for j in IntegerRange(N):
            assert pq.rhd_i0(i, j) == pq1.rhd_el(i, j)
            assert pq.rhd_i0(i, j) == pq1.rhd_i0(i, j)
            
    for i in IntegerRange(N):
        for n in IntegerRange(-N, N + 1):
            assert pq.pow_i0(i, n) == pq1.pow_el(i, n)
            assert pq.pow_i0(i, n) == pq1.pow_i0(i, n)

In [17]:
#test_memo_pq(Pq(get_gap_group(32, 17)))

In [19]:
Gg = gap.SmallGroup(16, 8)

In [22]:
Ggl = list(Gg.AsList()); Ggl

[<identity> of ...,
 f1,
 f2,
 f3,
 f4,
 f1*f2,
 f1*f3,
 f1*f4,
 f2*f3,
 f2*f4,
 f3*f4,
 f1*f2*f3,
 f1*f2*f4,
 f1*f3*f4,
 f2*f3*f4,
 f1*f2*f3*f4]

In [38]:
test_memo_pq(PqGap(gap.SmallGroup(24, 8)))

In [37]:
test_memo_pq(Pq(get_gap_group(24, 8)))

In [42]:
PqGap(gap.SmallGroup(24, 8)).create_precomputed_iso_pq()

In [43]:
Pq(get_gap_group(24, 8)).create_precomputed_iso_pq()

In [137]:
X = gap.SmallGroup(2187, 1000).MultiplicationTable()

In [143]:
Y = []

for i in range(1, 2187 + 1):
    Y.append([])
    for j in range(1, 2187 + 1):
        z = X[i][j].ToString()
        zs = int(z)
        Y[-1].append(zs)

ValueError: The gap session in which this object was defined is no longer running.

In [145]:
X = gap(1)

In [108]:
X.FamilyObj()

NewFamily( "CollectionsFamily(...)", [ 57 ], 
[ 56, 57, 99, 100, 101, 104, 108, 112, 116, 118, 119, 120, 121, 122, 123, 
  126, 130, 136, 148, 155, 158, 162, 216, 552, 553 ] )

In [120]:
Y = X.sage()

KeyboardInterrupt: Ctrl-c pressed while running Gap

In [106]:
len(X)

2187

In [116]:
type(X[0])

ValueError: The gap session in which this object was defined is no longer running.

In [96]:
Y0 = list(X)

In [97]:
Y1 = [list(t) for t in Y0]

KeyboardInterrupt: Ctrl-c pressed while running Gap

In [99]:
len(Y0[0])

RuntimeError: Gap produced error output
Error, Variable: '$sage2266' must have a value

   executing \$sage2266 = true;

In [95]:
type(Y[0])

<class 'sage.interfaces.gap.GapElement'>

In [24]:
Ggl.index(Ggl[1] * Ggl[2])

5

In [60]:
Y = X.sage()

KeyboardInterrupt: Ctrl-c pressed while running Gap

In [83]:
Y = [[int(a) for a in t] for t in X]

KeyboardInterrupt: Ctrl-c pressed while running Gap

In [53]:
X = gap.SmallGroup(64, 8).MultiplicationTable()

In [68]:
X.sage()

KeyboardInterrupt: Ctrl-c pressed while running Gap